In [44]:
import json
from datetime import datetime, time

from pymongo.mongo_client import MongoClient
import bson


* A partir du fichier trips_.json, remplacez les espaces des clés par un underscore, puis mettez en base en changeant les chaîne de caractères qui concerne des grandeurs numériques par le type integer ou float et les champs de date ($date) doit être sous format datetime.datetime. Ainsi les champs préfixé '$number...' suivent  l'exemple ci-dessous(cherchez du côté de la librairie bson, il existe une fonction utilitaire, il faut aussi réduire la profondeur):

```json
{'tripduration': {'$numberInt': "379"}}
```

en :

```json
{'tripduration': 379}
```



In [45]:
# On parse le fichie BSON avec la fonction bson.json_util.loads()
# Elle transformera les indices de type en type utilisable. 
# $numberInt -> int $date -> datetime, etc... 
# on peut les faire réapparaitre avec bson.json_util.dumps()
json_lines = []
with open("trips_.json", 'r') as file:
    for line in file:
        json_lines.append(bson.json_util.loads(line))

for i in range(3):
    print(json_lines[i])
    # print(bson.json_util.dumps(parsed_lines[i], json_options=bson.json_util.CANONICAL_JSON_OPTIONS))

{'tripduration': 307, 'start station id': 3118, 'start station name': 'McGuinness Blvd & Eagle St', 'end station id': 3119, 'end station name': 'Vernon Blvd & 50 Ave', 'bikeid': 23477, 'usertype': 'Subscriber', 'birth year': 1987, 'gender': 1, 'start station location': {'type': 'Point', 'coordinates': [-73.95284, 40.73555]}, 'end station location': {'type': 'Point', 'coordinates': [-73.95411749, 40.74232744]}, 'start time': datetime.datetime(2016, 1, 1, 7, 4, 31), 'stop time': datetime.datetime(2016, 1, 1, 7, 9, 38)}
{'tripduration': 3234, 'start station id': 500, 'start station name': 'Broadway & W 51 St', 'end station id': 337, 'end station name': 'Old Slip & Front St', 'bikeid': 20545, 'usertype': 'Customer', 'birth year': '', 'gender': 0, 'start station location': {'type': 'Point', 'coordinates': [-73.98336183, 40.76228826]}, 'end station location': {'type': 'Point', 'coordinates': [-74.00838676, 40.7037992]}, 'start time': datetime.datetime(2016, 1, 2, 8, 17, 50), 'stop time': dat

In [46]:
# Nettoyage des noms des clés avec python
parsed_lines = []
for line in json_lines:
    new_document = {}
    for k, v in line.items(): 
        new_document[k.replace(" ", "_")] = v
    parsed_lines.append(new_document)
    

# TODO: Nettoyage des noms des clé avec mongo. Si j'ai le temps

In [47]:
for i in range(3):
    print(parsed_lines[i])
    

{'tripduration': 307, 'start_station_id': 3118, 'start_station_name': 'McGuinness Blvd & Eagle St', 'end_station_id': 3119, 'end_station_name': 'Vernon Blvd & 50 Ave', 'bikeid': 23477, 'usertype': 'Subscriber', 'birth_year': 1987, 'gender': 1, 'start_station_location': {'type': 'Point', 'coordinates': [-73.95284, 40.73555]}, 'end_station_location': {'type': 'Point', 'coordinates': [-73.95411749, 40.74232744]}, 'start_time': datetime.datetime(2016, 1, 1, 7, 4, 31), 'stop_time': datetime.datetime(2016, 1, 1, 7, 9, 38)}
{'tripduration': 3234, 'start_station_id': 500, 'start_station_name': 'Broadway & W 51 St', 'end_station_id': 337, 'end_station_name': 'Old Slip & Front St', 'bikeid': 20545, 'usertype': 'Customer', 'birth_year': '', 'gender': 0, 'start_station_location': {'type': 'Point', 'coordinates': [-73.98336183, 40.76228826]}, 'end_station_location': {'type': 'Point', 'coordinates': [-74.00838676, 40.7037992]}, 'start_time': datetime.datetime(2016, 1, 2, 8, 17, 50), 'stop_time': dat

In [48]:
# Connexion à mongodb en localhost
connection = MongoClient() 
db = connection["operators"]
collection = db["trips"]


In [49]:
# on peut maintenant les insérer dans la base donnée.

collection.delete_many({}) 

collection.insert_many(parsed_lines)

InsertManyResult([ObjectId('688c7f47dfa3a3d123bbb339'), ObjectId('688c7f47dfa3a3d123bbb33a'), ObjectId('688c7f47dfa3a3d123bbb33b'), ObjectId('688c7f47dfa3a3d123bbb33c'), ObjectId('688c7f47dfa3a3d123bbb33d'), ObjectId('688c7f47dfa3a3d123bbb33e'), ObjectId('688c7f47dfa3a3d123bbb33f'), ObjectId('688c7f47dfa3a3d123bbb340'), ObjectId('688c7f47dfa3a3d123bbb341'), ObjectId('688c7f47dfa3a3d123bbb342'), ObjectId('688c7f47dfa3a3d123bbb343'), ObjectId('688c7f47dfa3a3d123bbb344'), ObjectId('688c7f47dfa3a3d123bbb345'), ObjectId('688c7f47dfa3a3d123bbb346'), ObjectId('688c7f47dfa3a3d123bbb347'), ObjectId('688c7f47dfa3a3d123bbb348'), ObjectId('688c7f47dfa3a3d123bbb349'), ObjectId('688c7f47dfa3a3d123bbb34a'), ObjectId('688c7f47dfa3a3d123bbb34b'), ObjectId('688c7f47dfa3a3d123bbb34c'), ObjectId('688c7f47dfa3a3d123bbb34d'), ObjectId('688c7f47dfa3a3d123bbb34e'), ObjectId('688c7f47dfa3a3d123bbb34f'), ObjectId('688c7f47dfa3a3d123bbb350'), ObjectId('688c7f47dfa3a3d123bbb351'), ObjectId('688c7f47dfa3a3d123bbb3

Après avoir mis en base et uniquement, assurez-vous qu'il n'y a pas de doublons, si il y en a, écrivez une fonction python permettant de les retirer (sans à avoir à faire de téléchargement)

In [50]:
# on detecte les doublons
duplicates = collection.aggregate([
    { "$group": {
        "_id": {
            "start_station_id": "$start_station_id", 
            "end_station_id": "$end_station_id", 
            "bikeid": "$bikeid", 
            "usertype": "$usertype", 
            "birth_year": "$birth_year", 
            "gender": "$gender", 
            "start_time": "$start_time", 
            "stop_time": "$stop_time"
        }, 
        "duplicate_count": {"$count": {}},
        "ids": {"$push": "$_id"} # on sauvegarde les ids pour les supprimer
    }},
    { "$match": { "duplicate_count" : {"$gt": 1}}},
    
    # Donne le nombre de total de doublons.
    # {"$group": { 
    #     "_id": "null", 
    #     "total": {"$sum": "$duplicate_count"}
    # }}
    # TODO: Trouver un moyen de faire un count sans obstruer le resultat de la requete
    
]).to_list()

total_duplicates = 0
for duplicate in duplicates:
    total_duplicates += duplicate["duplicate_count"]
    
print(f'il y a {total_duplicates} doublons')

il y a 31 doublons


In [51]:
# On les supprime les doublons 
deleted_count = 0
for duplicate in duplicates:
    res = collection.delete_many({"_id": { "$in": duplicate["ids"][1:]}}) # Tous sauf le premier
    deleted_count += res.deleted_count

print(f'{deleted_count} doublons ont été supprimés')

19 doublons ont été supprimés


In [52]:
# On devrait avoir 10000 documents
count = collection.aggregate([
    { "$count" : "document_count"}
])

print(count.next()['document_count'])

10000


De plus, des hypothèses d'incohérences ont été émises par les parties prenantes, il s'agit d'écrire pour chacune de ces hypothèses une fonction python qui va détecter les documents incriminés et les flagger avec un champs supplémentaire pour rectification/enquête ultérieure :

* Un vélo loué deux fois, mais la deuxième période de location démarre avant le rendu de la première période de location, ce qui n'est pas normal, ou tout autre chevanchement de période.
* Un utilisateur trop jeune (le service est réservé au + de 13 ans)
* La date de naissance n'est pas renseignée
* Des locations trop courtes (1 secondes)
* Des temps de location incohérents par rapport au start_time et au end_time


Faites un rapport détaillé des anomalies trouvées (nombres d'occurences)

In [53]:
# Velo loué plusieurs fois en même temps

res = collection.aggregate([
    {"$group": {
        "_id": "$bikeid", 
        "trip_count": {"$count": {}},
        "trips": {"$push": {
            "_id": "$_id",
            "start": "$start_time",
            "stop": "$stop_time"
        }}
    }},
    { "$match": {"trip_count": {"$gt": 1}}},
    { "$project": {
        "_id": 1,
        "trips": {"$sortArray": {"input": "$trips", "sortBy": {"start" : 1} }}
    }}
]).to_list()

trips_rented_before_end = set()
for bike in res:
    for i in range(len(bike['trips']) - 1):
        if bike["trips"][i]["stop"] > bike["trips"][i+1]["start"]:
            trips_rented_before_end.update([bike["trips"][i]["_id"],  bike["trips"][i+1]["_id"]])


res = collection.update_many(
    {"_id": {"$in": list(trips_rented_before_end)}}, 
    {"$push": {"anomaly": "bike_rented_before_end"}}
)

print(f'Documents flaggés "bike_rented_before_end": {res.modified_count}')


Documents flaggés "bike_rented_before_end": 3


In [54]:
# Utilisateur trop jeune (-13 ans)

res = collection.update_many(
    {
        "birth_year": {"$type": "int"},
        "$expr": { 
            "$gt": [ "$birth_year", { "$subtract": [ {"$year": "$$NOW"}, 13] } ]
        },
    }, 
    { 
        "$push": { "anomaly": "below_13_years_old" }
    }
)    

print(f'Documents flaggés "below_13_years_old": {res.modified_count}')

Documents flaggés "below_13_years_old": 2


In [55]:
# Date de naissance manquante

birth_field_data = collection.aggregate([
    {"$group": {
        "_id": {"$type": "$birth_year"},
        "total": {"$count": {}}
    }}
]).to_list()
print(birth_field_data)

strings_values= collection.aggregate([
    {"$match": { 
        "$expr": { "$eq" : [
            {"$type": "$birth_year"}, 
            "string"
        ]}
    }},
    {"$group": {
        "_id": "$birth_year",
        "count": { "$count": {}}
    }}
])
print(strings_values.to_list())

int_values = collection.aggregate([
    {"$match": { 
        "$expr": { "$eq" : [
            {"$type": "$birth_year"}, 
            "int"
        ]}
    }},
    {"$group": {
        "_id": "$birth_year",
        "count": { "$count": {}}
    }},
    {"$sort" : {"_id": 1}}
])
print(int_values.to_list())

res = collection.update_many(
    {
        "$or": [
            {"birth_year": {"$eq": ""}},
            {"birth_year": {"$eq": 0}},
            {"birth_year": {"$eq": "null"}},
        ]
    }, 
    { "$push": {"anomaly": "missing_birth_year"}}
)

print(f'Documents flaggés "missing_birth_year": {res.modified_count}')


[{'_id': 'int', 'total': 8011}, {'_id': 'string', 'total': 1989}]
[{'_id': '', 'count': 1989}]
[{'_id': 1885, 'count': 3}, {'_id': 1936, 'count': 2}, {'_id': 1938, 'count': 6}, {'_id': 1939, 'count': 2}, {'_id': 1940, 'count': 6}, {'_id': 1941, 'count': 9}, {'_id': 1942, 'count': 16}, {'_id': 1943, 'count': 3}, {'_id': 1944, 'count': 10}, {'_id': 1945, 'count': 10}, {'_id': 1946, 'count': 19}, {'_id': 1947, 'count': 25}, {'_id': 1948, 'count': 24}, {'_id': 1949, 'count': 28}, {'_id': 1950, 'count': 38}, {'_id': 1951, 'count': 40}, {'_id': 1952, 'count': 52}, {'_id': 1953, 'count': 71}, {'_id': 1954, 'count': 85}, {'_id': 1955, 'count': 92}, {'_id': 1956, 'count': 96}, {'_id': 1957, 'count': 106}, {'_id': 1958, 'count': 115}, {'_id': 1959, 'count': 127}, {'_id': 1960, 'count': 150}, {'_id': 1961, 'count': 125}, {'_id': 1962, 'count': 153}, {'_id': 1963, 'count': 163}, {'_id': 1964, 'count': 99}, {'_id': 1965, 'count': 145}, {'_id': 1966, 'count': 122}, {'_id': 1967, 'count': 127}, {'_id

In [56]:
# locations trops courtes (1 seconde)
res = collection.update_many(
    {
        "$expr": { "$lte": [
            {"$subtract": [ "$stop_time", "$start_time" ]}, 
            1
        ]}
    }, {
        "$push": {"anomaly": "short_rent"}
    }
)

print(f'Document flaggés "short_rent": {res.modified_count}')

Document flaggés "short_rent": 4


In [57]:
# Temps de location incohérents (stop_time < start_time)

# ------DEBUGGING utility
# res = collection.aggregate([
#     {
#         "$project": {
#             "tripduration": {"$multiply": ["$tripduration", 1000]},
#             "elapsed": {"$subtract": ["$stop_time", "$start_time"]},
#         }
#     },
#     {
#         "$project": {
#             "dur": {"$abs": {"$subtract": ["$tripduration", "$elapsed"]}},
#         }
#     },
#     {
#         "$match": {
#             "dur": {"$gt": 2000}
#         }
#     }
# ]).to_list()
#
# for i in range(20):
#     print(res[i])

res = collection.update_many(
    {"$expr": {
        "$gt": [
            {
                "$abs": {
                    "$subtract": [
                        {"$multiply": ["$tripduration", 1000]},
                        {"$subtract": ["$stop_time", "$start_time"]}
                    ]}
            },
            1000 # Tolérance en ms
        ]
    }},
    {"$push": {"anomaly": "tripduration_mismatch"}}
)

print(f'Document flaggés "tripduration_mismatch": {res.modified_count}')

Document flaggés "tripduration_mismatch": 10


In [65]:
# Bilan des documents flaggés
res = collection.aggregate([
    {"$match": {"anomaly": {"$exists": True}}},
    {"$unwind": "$anomaly"},
    {"$group": {
        "_id": "$anomaly",
        "count": {"$count": {}}
    }}    
]).to_list()

print("Détails des anomalies:")
total = 0
for anomaly in res:
    total += anomaly["count"]
    print(f'{anomaly["_id"]}: {anomaly["count"]}')
    
print(f"{total} anomalies trouvées")  

Détails des anomalies:
bike_rented_before_end: 3
short_rent: 4
missing_birth_year: 1989
tripduration_mismatch: 10
below_13_years_old: 2
2008 anomalies trouvées


Puis, après avoir fait un nettoyage et écarté les locations suspectes, répondez aux questions métier qui suivent :

* Changez le gender 0 en gender 1

* Quels sont les 5 trajets (start station → end station) les plus fréquents pour les utilisateurs de genre féminin ? (gender = 1)

* Quel est le nombre total de trajets par type d’utilisateur (Subscriber vs Customer) pour l’année 2023 ?

* Quelle est la durée moyenne des trajets par station de départ pour les trajets commençant entre 7h et 9h ?

* Quel est le top 3 des stations avec la plus forte augmentation du nombre de trajets entre deux années consécutives ?

* Quelle est la durée médiane des trajets pour les + de 65 ans ?

* Quelle est la répartition des trajets (nombre de trajets) par tranche horaire de 2 heures (faire visualisation, 0h-2h, 2h-4h etc..) ?

* Quel est le temps moyen passé en trajet pour chaque genre, filtré sur les trajets de plus de 10 minutes ?

* Combien de trajets ont démarré pour chaque station pendant les heures de pointe (ex. 7h-9h et 17h-19h) ?

* Quel est l'âge le plus courant pour les locations de 18h à 20h ?